In [ ]:
from glob import glob
import os

import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
from matplotlib import cm
import geopandas as gpd
import rioxarray as rxr
import xarray as xr
from rasterio.plot import plotting_extent
import h5py
from shapely.geometry import box

from osgeo import gdal

# optional - turn off warnings
import warnings
warnings.filterwarnings('ignore')

from constants import *
os.chdir(home_dir)

In [ ]:
### read in SWE data and save as npy file
'''
years = range(1987,2017)
months = ['Oct', 'Nov', 'Dec','Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep']
m_num = range(1,13)
days_in_month = [31, 30, 31, 31, 28, 31, 30, 31, 30, 31, 31, 30]
os.chdir('/users/marianne/documents/data/reanalysis')
for yr in years:
    hf = h5py.File('SN_SWE_WY'+ str(yr) + '.h5', 'r')
    swe = hf.get('SWE')
    lat = hf.get('lat')
    lon = hf.get('lon')
    for i in range(len(days_in_month)):
        start = sum(days_in_month[0:i-1])
        end = start + days_in_month[i]
        tmp = swe[start:end,0:2850,:]
        tmp = tmp.astype('float')
        for a in range(days_in_month[i]):
            for b in range(2850):
                for c in range(6601):
                    try:
                        if tmp[a,b,c]<0:
                            tmp[a,b,c] = np.nan
                    except:
                        continue
        da = xr.DataArray(
                data = np.nanmean(tmp,axis=0),
                dims = ["lon","lat"],
                coords = dict(
                    lon=(lon[:2850,0]),
                    lat=(lat[0,:]),
                )
            )
        np.save('swe_'+str(yr)+'_'+months[i]+'_N.npy',da,allow_pickle=True)

    for i in range(len(days_in_month)):
        start = sum(days_in_month[0:i-1])
        end = start + days_in_month[i]
        tmp = swe[start:end,2850:,:]
        tmp = tmp.astype('float32')/1000
        for a in range(days_in_month[i]):
            for b in range(2850,2850*2):
                for c in range(6601):
                    try: 
                        if tmp[a,b,c]<0:
                            tmp[a,b,c] = np.nan
                    except:
                        continue
        da = xr.DataArray(
                data = np.nanmean(tmp,axis=0),
                dims = ["lon","lat"],
                coords = dict(
                    lon=(lon[2850:,0]),
                    lat=(lat[0,:]),
                )
            )
        np.save('swe_'+str(yr)+'_'+months[i]+'_S.npy',da,allow_pickle=True)
'''

In [ ]:
### getr max swe for each year and save

fp = 'c:/users/marianne/documents/data/reanalysis/h5/'
# daily_swe = {}
daily_swe_total = {}
max_day = []
max_swe = []
for yr in range(1985 , 2016):
# for yr in [2000]:
    hf = h5py.File(fp + 'SN_SWE_WY'+ str(yr) + '.h5', 'r')
    swe = hf.get('SWE')
    dailyswe = []
    numpixels = []
    for i in range(365):
        swe_tmp = np.array(swe[i,:,:])/1000000
        swe_tmp=swe_tmp[swe_tmp>0]
        dailyswe.append(np.nansum(swe_tmp))
        numpixels.append(len(swe_tmp))
    #dailyswe=np.array(dailyswe)
    numpixels = np.array(numpixels)
    maxswe = np.nanmax(dailyswe[numpixels>1e6])
    maxday = np.where(dailyswe == maxswe)[0][0]
    daily_swe_total[yr] = dailyswe
    max_day.append(maxday)
    max_swe.append(maxswe)


## save to npy files in home directory
np.save('daily_swe_sum.npy',daily_swe_total)
np.save('max_day.npy',max_day)
np.save('max_swe.npy',max_swe)


In [ ]:
import h5py
max_swe_map = {}
i=0
for yr in range(1985,2016):
    file = 'data/data'+str(yr)+'.npy'
    data =np.load(file,allow_pickle=True).reshape((1))[0]
    hf = h5py.File('/users/marianne/documents/data/reanalysis/h5/SN_SWE_WY'+ str(yr) + '.h5', 'r')
    swe = hf.get('SWE')
    idx = max_day[i]
    i=i+1
    tmp = swe[idx,:,:]
    max_swe_map[yr] = tmp
np.save('data/maxs_we_map.npy',max_swe_map)

In [ ]:
## save data as lat lon
days = range(90,200,10)
for yr in range(1985,2017):
    hf = h5py.File(filepath + 'SN_SWE_WY'+str(yr)+ '.h5', 'r')
    swe = hf.get('SWE')
    lat = hf.get('lat')
    lon = hf.get('lon')
    data = {}
    for i in days:
        tmp = np.array(swe[i,:,:]).flatten()
        tmp = tmp[np.where((tmp>0) & (tmp<2000))]
        data[i]=tmp
    np.save('data'+str(yr)+'.npy',data,allow_pickle=True)

In [ ]:
## aggregate monthly PE, NDVI, etc.